## Importing the Libraries


In [1]:
import numpy as np
import pandas as pd

## Importing the dataset


In [2]:
dataset = pd.read_csv('../Data/Groceries_dataset.csv')

print('No of rows & columns:- \n', dataset.describe)
print('First set of rows in dataset:- \n', dataset.head())

No of rows & columns:- 
 <bound method NDFrame.describe of        Member_number        Date        itemDescription
0               1808  21-07-2015         tropical fruit
1               2552  05-01-2015             whole milk
2               2300  19-09-2015              pip fruit
3               1187  12-12-2015       other vegetables
4               3037  01-02-2015             whole milk
...              ...         ...                    ...
38760           4471  08-10-2014          sliced cheese
38761           2022  23-02-2014                  candy
38762           1097  16-04-2014               cake bar
38763           1510  03-12-2014  fruit/vegetable juice
38764           1521  26-12-2014               cat food

[38765 rows x 3 columns]>
First set of rows in dataset:- 
    Member_number        Date   itemDescription
0           1808  21-07-2015    tropical fruit
1           2552  05-01-2015        whole milk
2           2300  19-09-2015         pip fruit
3           1187  12-

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


## Group this dataset according to Member_number & Date


In [4]:
transactions = dataset.groupby(['Member_number','Date'])
transactions.count()

itemDescription
Member_number Date                       
1000          15-03-2015                4
              24-06-2014                3
              24-07-2015                2
              25-11-2015                2
              27-05-2015                2
...                                   ...
4999          24-01-2015                6
              26-12-2015                2
5000          09-03-2014                2
              10-02-2015                3
              16-11-2014                2

[14963 rows x 1 columns]

In [5]:
transactions_items = []

for transaction in list(transactions):
    details = transaction[1]['itemDescription'].tolist()
    transactions_items.append(details)

In [6]:
transactions_items[: 4]

[['sausage', 'whole milk', 'semi-finished bread', 'yogurt'],
 ['whole milk', 'pastry', 'salty snack'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles']]

## Training the Apriori model on the dataset


In [7]:
from apyori import apriori

rules = apriori(transactions_items, min_support = 0.001, min_confidance = 0.05, min_lift = 1.2, min_length = 2, max_length = 2)

# Visualizing Results

In [8]:
results = list(rules)
print(results)

[RelationRecord(items=frozenset({'beef', 'brown bread'}), support=0.0015371249081066632, ordered_statistics=[OrderedStatistic(items_base=frozenset({'beef'}), items_add=frozenset({'brown bread'}), confidence=0.045275590551181105, lift=1.2033013524286376), OrderedStatistic(items_base=frozenset({'brown bread'}), items_add=frozenset({'beef'}), confidence=0.04085257548845471, lift=1.2033013524286376)]), RelationRecord(items=frozenset({'frozen vegetables', 'beef'}), support=0.0012697988371315912, ordered_statistics=[OrderedStatistic(items_base=frozenset({'beef'}), items_add=frozenset({'frozen vegetables'}), confidence=0.03740157480314961, lift=1.3356557608103283), OrderedStatistic(items_base=frozenset({'frozen vegetables'}), items_add=frozenset({'beef'}), confidence=0.045346062052505964, lift=1.3356557608103283)]), RelationRecord(items=frozenset({'beef', 'margarine'}), support=0.001403461872619127, ordered_statistics=[OrderedStatistic(items_base=frozenset({'beef'}), items_add=frozenset({'mar

## Putting the results well organised into a Pandas DataFrame

In [9]:
def inspect(results):
    lhs         = [tuple(result[2][0][0])[0] for result in results]
    rhs         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(lhs, rhs, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['Left Hand Side', 'Right Hand Side', 'Support', 'Confidence', 'Lift'])

In [10]:
resultsinDataFrame

,Left Hand Side,Right Hand Side,Support,Confidence,Lift
0,beef,brown bread,0.001537,0.045276,1.203301
1,beef,frozen vegetables,0.001270,0.037402,1.335656
2,beef,margarine,0.001403,0.041339,1.283297
3,beef,newspapers,0.001671,0.049213,1.265237
4,beverages,sausage,0.001537,0.092742,1.536764
5,bottled beer,sausage,0.003342,0.073746,1.222000
6,bottled water,sugar,0.001470,0.024229,1.368074
7,brown bread,canned beer,0.002406,0.063943,1.362937
8,butter,margarine,0.001403,0.039848,1.237030
9,candy,citrus fruit,0.001002,0.069767,1.313120
